<a href="https://colab.research.google.com/github/JValdez777/asl-translator-notebook/blob/main/aiASLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print("CUDA OK:", torch.cuda.is_available())

CUDA OK: True
